In [1]:
%load_ext autoreload

In [1]:
import sys
import os

# Get the project's root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Change the current working directory to project_root
os.chdir(project_root)

print(os.getcwd())  # This should now print the project_root path



c:\Users\David\Source\supervisedgenksc


In [2]:
import torch
import torch.nn as nn

from src.training import RKMStiefelLightning, load_config




batch_size = 4
num_clusters = 3

# Create a dummy distance matrix (batch_size x num_clusters)
# Each row represents distances from a sample to each cluster
dcos = torch.tensor([
    [0.8, 0.3, 0.5],  # Sample 0 distances to clusters 0,1,2
    [0.4, 0.1, 0.6],  # Sample 1 distances to clusters 0,1,2
    [0.7, 0.9, 0.2],  # Sample 2 distances to clusters 0,1,2
    [0.5, 0.4, 0.3]   # Sample 3 distances to clusters 0,1,2
], dtype=torch.float32)

# Create labels: some labeled, some unlabeled (NaN)
# 0 -> label is 0, 1 -> label is 1, NaN -> unlabeled
labels = torch.tensor([0, 1, float('nan'), 2], dtype=torch.float32)


config = load_config("config/config.yaml")

final_dcos, loss = RKMStiefelLightning(config=config)._calculate_cosine_distance_loss(dcos=dcos, labels=labels,N= dcos.size(0))

print("Original dcos matrix:")
print(dcos)
print("\nLabels (NaN means unlabeled):")
print(labels)
print("\nFinal dcos (distances used for loss calculation):")
print(final_dcos)
print("\nCosine distance loss:", loss)

# Expected values calculation
expected_final_dcos = torch.tensor([
    0.8,  # Sample 0 has label 0, so distance is 0.8
    0.1,  # Sample 1 has label 1, so distance is 0.1
    0.2,  # Sample 2 is unlabeled, min distance is 0.2
    0.3   # Sample 3 has label 2, so distance is 0.3
], dtype=torch.float32)

expected_loss = expected_final_dcos.sum() / batch_size

print("\nExpected final dcos:")
print(expected_final_dcos)
print("Expected loss:", expected_loss.item())


Original dcos matrix:
tensor([[0.8000, 0.3000, 0.5000],
        [0.4000, 0.1000, 0.6000],
        [0.7000, 0.9000, 0.2000],
        [0.5000, 0.4000, 0.3000]])

Labels (NaN means unlabeled):
tensor([0., 1., nan, 2.])

Final dcos (distances used for loss calculation):
tensor([[0.8000, 0.1000, 0.3000],
        [0.8000, 0.1000, 0.3000],
        [0.2000, 0.2000, 0.2000],
        [0.8000, 0.1000, 0.3000]])

Cosine distance loss: tensor([[0.6500, 0.1250, 0.2750]])

Expected final dcos:
tensor([0.8000, 0.1000, 0.2000, 0.3000])
Expected loss: 0.3500000238418579


In [12]:
N = dcos.size(0)
k = dcos.size(1)

final_dcos = torch.zeros_like(labels)
unlabelled_mask = torch.isnan(labels)
labelled_mask = ~unlabelled_mask



if labelled_mask.any():
    final_dcos[labelled_mask] = dcos[labelled_mask, labels[labelled_mask].long()]

if unlabelled_mask.any():
    final_dcos[unlabelled_mask] = torch.min(dcos[unlabelled_mask], dim=1).values

print(f"Labels : {labels}")
print(f"Distances : {dcos}")

print(f"Final Distances : {final_dcos}")


Labels : tensor([0., 1., nan, 2.])
Distances : tensor([[0.8000, 0.3000, 0.5000],
        [0.4000, 0.1000, 0.6000],
        [0.7000, 0.9000, 0.2000],
        [0.5000, 0.4000, 0.3000]])
Final Distances : tensor([0.8000, 0.1000, 0.2000, 0.3000])


In [ ]:

if unlabelled_mask.any():
    final_dcos[unlabelled_mask] = torch.min(dcos[unlabelled_mask], dim=1).values

oneN = torch.ones(N, 1)
cosine_distance_loss = oneN.t() @ final_dcos